# Get started with Redshift and the Feature Store

This tutorial notebook will help you get started with working with the Hopsworks feature store and Redshift.


The tutorial is divided in 2 parts: 
* [Create a sample Amazon Redshift cluster](#setup_redhsift)
* [Load sample data into Redshift cluster](#load_data_redhsift)

## Create a sample Amazon Redshift cluster<a name="setup_redhsift"></a>

##### This step will describe how to  Create a sample Amazon Redshift cluster. If you already have Redshift cluster you may skip this step 

To setup sample redshift cluster please follow the steps [here](https://docs.aws.amazon.com/redshift/latest/gsg/rs-gsg-launch-sample-cluster.html)

To to intrecat with redshift cluser from hopsworks modify your redshift cluster's follow steps bellow:


1) Make sure that your EC2 instance, that hopsworks runs on, has AIM role with Redshift access policies 
2) Make sure that your redshift cluster's inbound traffic rules allowe access from hopsworks EC2 instance:
     * From AWS management console go to VPC
     * From VPC go to Security Groups
     * Create new secuiry group or if you have already created modify Inbound rules:
         a) go to Inbound rules 
         b) Edit inbound rules
         c) Add rule
         e) In Type section select Redshift
         f) In Source section select Custom
         g) Then add name of security group of your hopsworks EC2 instance 
         
3) [Download an Amazon Redshift JDBC driver](https://docs.aws.amazon.com/redshift/latest/mgmt/configure-jdbc-connection.html#download-jdbc-driver)

## Load  sample data into Redshift cluster <a name="load_data_redhsift"></a>

### Import a CSV in Redshift from s3 bucket

Importing a CSV into Redshift requires you to create a table first. 

<code>
    CREATE TABLE telcom (
        customer_id VARCHAR primary key 
        gender VARCHAR,   
        senior_citizen VARCHAR, 
        partner VARCHAR,
        dependents VARCHAR,  
        tenure INTEGER, 
        phone_service VARCHAR,      
        multiple_lines VARCHAR, 
        internet_service VARCHAR, 
        online_security VARCHAR, 
        online_backup VARCHAR, 
        device_protection VARCHAR, 
        tech_support VARCHAR, 
        streaming_tv VARCHAR,
        streaming_movies VARCHAR,        
        contract VARCHAR,
        paperless_billing VARCHAR,            
        payment_method INTEGER, 
        monthly_charges INTEGER, 
        total_charges INTEGER, 
        churn VARCHAR    
    );
</code>

and then copy

<code>
    COPY telcom
        FROM 's3://<your-bucket-name>/load/file_name.csv'
        credentials 'aws_access_key_id=<Your-Access-Key-ID>;aws_secret_access_key=<Your-Secret-Access-Key>'
    CSV;
</code>
    
please refer to the [Redshift COPY Command Specification](https://docs.aws.amazon.com/redshift/latest/dg/r_COPY.html) for a complete list of options for COPY,     

    
### Import a telcom data in Redshift from hopsworks

In [ ]:
import org.apache.spark.api.java.JavaSparkContext;
import org.apache.spark.sql.DataFrameWriter;
import org.apache.spark.sql.Dataset;
import org.apache.spark.sql.Row;
import org.apache.spark.sql.SaveMode;
import org.apache.spark.sql.SparkSession;
import io.hops.util.Hops
import org.apache.spark.sql._
import spark.implicits._
import org.apache.spark.sql.types._

In [ ]:
val jdbcUsername = "YOUR_REDSHIFT_USER_NAME"
val AIMrole = "AIM_name_of_EC2_with_redshift_access"
val jdbcHostname = "redshift-cluster-1.citpxgaovgkr.eu-north-1.redshift.amazonaws.com"
val jdbcPort = 5439
val jdbcDatabase = "telcom"
//val jdbcUrl = s"jdbc:redshift://${jdbcHostname}:${jdbcPort}/${jdbcDatabase}"
val jdbcUrl = s"jdbc:redshift:iam://${jdbcHostname}:${jdbcPort}/${jdbcDatabase}"

In [ ]:
telcom = spark.read.csv("path")

telcom
  write.
  format("jdbc").
  option("driver", "com.amazon.redshift.jdbc42.Driver").
  option("url",jdbcUrl).
  option("dbtable", jdbcDatabase).
  option("user", jdbcUsername).
  option("aws_iam_role", AIMrole).
  mode("append"). 
  save()